In [1]:
import os
import numpy as np
import pandas as pd
from astroquery.skyview import SkyView
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt


In [4]:
# Load previously saved grouped YSO data
grouped_path = r"C:\Users\kgokh\Downloads\TAMOONS\grouped_ysos_IC348.csv"
df = pd.read_csv(grouped_path)

source_name = grouped_path.replace(r"C:\Users\kgokh\Downloads\TAMOONS\grouped_ysos_", "").replace(".csv", "")
print(source_name)  

IC348


In [ ]:
# Create a list of subgroups
final_sublists = [group.reset_index(drop=True) for _, group in df.groupby('Group')]

# Compute global sky center
ra_global_median = df['RA_deg'].median()
dec_global_median = df['DEC_deg'].median()
median_coord = SkyCoord(ra=ra_global_median*u.deg, dec=dec_global_median*u.deg)


In [ ]:
# Create output folders for FITS images and previews
main_dir = os.getcwd()
fits_dir = os.path.join(main_dir, f"fits_images_{source_name}")
preview_dir = os.path.join(main_dir, f"fits_previews_{source_name}")
os.makedirs(fits_dir, exist_ok=True)
os.makedirs(preview_dir, exist_ok=True)

# Initialize list 
group_summaries = []


In [ ]:
# Loop through each group and fetch its FITS image
for i, group in enumerate(final_sublists, 1):
    group_coords = SkyCoord(ra=group['RA_deg'].values*u.deg, dec=group['DEC_deg'].values*u.deg)

    # Compute the group’s center position
    ra_mean = group['RA_deg'].mean()
    dec_mean = group['DEC_deg'].mean()
    mean_coord = SkyCoord(ra=ra_mean*u.deg, dec=dec_mean*u.deg)

    # Find farthest offset from the mean
    offsets = [mean_coord.separation(c).arcminute for c in group_coords]
    max_offset = max(offsets)

    # Replace center if too close to each other (fallback to global)
    if max_offset < 0.3:
        ra_used, dec_used = ra_global_median, dec_global_median
        coord_used = median_coord
        print(f"🔁 Group {i} center replaced with global median (too close)")
    else:
        ra_used, dec_used = ra_mean, dec_mean
        coord_used = mean_coord

    group_summaries.append({
        'Group': i,
        'N_Targets': len(group),
        'RA_center': ra_used,
        'DEC_center': dec_used
    })

    # Try fetching DSS image for this group
    try:
        print(f" Downloading Group {i} image...")
        images = SkyView.get_images(position=coord_used, survey=['DSS'], radius=0.1 * u.deg)
        if not images:
            print(f"⚠️ No FITS found for Group {i}")
            continue

        fits_path = os.path.join(fits_dir, f"group_{i}_{source_name}.fits")
        images[0].writeto(fits_path, overwrite=True)

        # Read FITS and set up WCS
        hdul = fits.open(fits_path)
        hdu = hdul[0]
        wcs = WCS(hdu.header)
        data = hdu.data


In [ ]:
        # Generate image with proper scaling
        plt.figure(figsize=(6, 6))
        try:
            vmin = np.nanpercentile(data, 5)
            vmax = np.nanpercentile(data, 95)
        except:
            vmin, vmax = np.nanmin(data), np.nanmax(data)

        plt.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)

        # Center pixel
        center_x, center_y = wcs.world_to_pixel(coord_used)

        for _, star in group.iterrows():
            star_coord = SkyCoord(star['RA_deg'], star['DEC_deg'], unit='deg')
            xpix, ypix = wcs.world_to_pixel(star_coord)

            dx_arcmin = (xpix - center_x) * abs(wcs.wcs.cdelt[0]) * 60
            dy_arcmin = (ypix - center_y) * abs(wcs.wcs.cdelt[1]) * 60
            offset = np.sqrt(dx_arcmin**2 + dy_arcmin**2)

            plt.plot(xpix, ypix, 'ro', markersize=5)
            plt.text(xpix + 5, ypix + 5, f"{offset:.2f}\"", color='yellow', fontsize=8)


In [ ]:
        # Setup axes with ΔRA and ΔDEC
        x_ticks = np.linspace(-6, 6, 5)
        y_ticks = np.linspace(-6, 6, 5)
        arcmin_per_pix = abs(wcs.wcs.cdelt[0]) * 60
        xticks_pix = center_x + x_ticks / arcmin_per_pix
        yticks_pix = center_y + y_ticks / arcmin_per_pix

        plt.xticks(xticks_pix, [f"{x:.1f}" for x in x_ticks])
        plt.yticks(yticks_pix, [f"{y:.1f}" for y in y_ticks])

        plt.xlabel("ΔRA (arcmin)")
        plt.ylabel("ΔDEC (arcmin)")
        plt.title(f"Group {i} - DSS")
        plt.grid(True)
        plt.tight_layout()

        preview_path = os.path.join(preview_dir, f"group_{i}_preview_{source_name}.png")
        plt.savefig(preview_path, bbox_inches='tight')
        plt.close()

        print(f" Group {i} preview saved")

    except Exception as e:
        print(f" Error Group {i}: {e}")


In [ ]:
# Save the final summary CSV with group centers
summary_df = pd.DataFrame(group_summaries)
summary_out = os.path.join(main_dir, f"group_summary_{source_name}.csv")
summary_df.to_csv(summary_out, index=False)

print(f"\n📊 Group summary saved at: {summary_out}")
